# Bayesian Augmented Tensor Factorization

**Published**: November 12, 2020

**Author**: Yixian Chen [[**GitHub homepage**](https://github.com/yxnchen)], Xinyu Chen [[**GitHub homepage**](https://github.com/xinychen)]

**Download**: This Jupyter notebook is at our GitHub repository. If you want to evaluate the code, please download the notebook from the [**transdim**](https://github.com/xinychen/transdim/blob/master/imputer/BATF.ipynb) repository.

This notebook shows how to implement the Bayesian Augmented Tensor Factorization (BATF) model on some real-world data sets. In the following, we will discuss:

- What the BATF is.

- How to implement BATF mainly using Python `numpy` with high efficiency.

- How to make imputation on some real-world spatiotemporal datasets.

To overcome the problem of missing values within multivariate time series data, this model takes into account low-rank tensor structure by folding data along day dimension. For an in-depth discussion of BATF, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Zhaocheng He, Yixian Chen, Yuhuan Lu, Jiawei Wang (2019). <b>Missing traffic data imputation and pattern discovery with a Bayesian augmented tensor factorization model</b>. Transportation Research Part C: Emerging Technologies, 104: 66-77. <a href="https://doi.org/10.1016/j.trc.2019.03.003" title="PDF"><b>[PDF]</b></a> 
</font>
</div>

We start by importing the necessary dependencies. We will make use of `numpy` and `scipy`.

In [1]:
import numpy as np
from numpy.random import multivariate_normal as mvnrnd
from scipy.stats import wishart
from numpy.random import normal as normrnd
from scipy.linalg import khatri_rao as kr_prod
from numpy.linalg import inv as inv
from numpy.linalg import solve as solve
from numpy.linalg import cholesky as cholesky_lower
from scipy.linalg import cholesky as cholesky_upper
from scipy.linalg import solve_triangular as solve_ut

In [2]:
def mvnrnd_pre(mu, Lambda):
    src = normrnd(size = (mu.shape[0],))
    return solve_ut(cholesky_upper(Lambda, overwrite_a = True, check_finite = False), 
                    src, lower = False, check_finite = False, overwrite_b = True) + mu

### CP decomposition

#### CP Combination (`cp_combine`)

- **Definition**:

The CP decomposition factorizes a tensor into a sum of outer products of vectors. For example, for a third-order tensor $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$, the CP decomposition can be written as

$$\hat{\mathcal{Y}}=\sum_{s=1}^{r}\boldsymbol{u}_{s}\circ\boldsymbol{v}_{s}\circ\boldsymbol{x}_{s},$$
or element-wise,

$$\hat{y}_{ijt}=\sum_{s=1}^{r}u_{is}v_{js}x_{ts},\forall (i,j,t),$$
where vectors $\boldsymbol{u}_{s}\in\mathbb{R}^{m},\boldsymbol{v}_{s}\in\mathbb{R}^{n},\boldsymbol{x}_{s}\in\mathbb{R}^{f}$ are columns of factor matrices $U\in\mathbb{R}^{m\times r},V\in\mathbb{R}^{n\times r},X\in\mathbb{R}^{f\times r}$, respectively. The symbol $\circ$ denotes vector outer product.

- **Example**:

Given matrices $U=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]\in\mathbb{R}^{2\times 2}$, $V=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ 5 & 6 \\ \end{array} \right]\in\mathbb{R}^{3\times 2}$ and $X=\left[ \begin{array}{cc} 1 & 5 \\ 2 & 6 \\ 3 & 7 \\ 4 & 8 \\ \end{array} \right]\in\mathbb{R}^{4\times 2}$, then if $\hat{\mathcal{Y}}=\sum_{s=1}^{r}\boldsymbol{u}_{s}\circ\boldsymbol{v}_{s}\circ\boldsymbol{x}_{s}$, then, we have

$$\hat{Y}_1=\hat{\mathcal{Y}}(:,:,1)=\left[ \begin{array}{ccc} 31 & 42 & 65 \\ 63 & 86 & 135 \\ \end{array} \right],$$
$$\hat{Y}_2=\hat{\mathcal{Y}}(:,:,2)=\left[ \begin{array}{ccc} 38 & 52 & 82 \\ 78 & 108 & 174 \\ \end{array} \right],$$
$$\hat{Y}_3=\hat{\mathcal{Y}}(:,:,3)=\left[ \begin{array}{ccc} 45 & 62 & 99 \\ 93 & 130 & 213 \\ \end{array} \right],$$
$$\hat{Y}_4=\hat{\mathcal{Y}}(:,:,4)=\left[ \begin{array}{ccc} 52 & 72 & 116 \\ 108 & 152 & 252 \\ \end{array} \right].$$

In [3]:
def cp_combine(var):
    return np.einsum('is, js, ts -> ijt', var[0], var[1], var[2])

In [4]:
factor = [np.array([[1, 2], [3, 4]]), np.array([[1, 3], [2, 4], [5, 6]]), 
          np.array([[1, 5], [2, 6], [3, 7], [4, 8]])]
print(cp_combine(factor))
print()
print('tensor size:')
print(cp_combine(factor).shape)

[[[ 31  38  45  52]
  [ 42  52  62  72]
  [ 65  82  99 116]]

 [[ 63  78  93 108]
  [ 86 108 130 152]
  [135 174 213 252]]]

tensor size:
(2, 3, 4)


### Vector combination (`vec_combine`)

In [5]:
## 1st solution
def vec_combine(vector):
    tensor = 0
    d = len(vector)
    for i in range(d):
        ax = [len(vector[i]) if j == i else 1 for j in range(d)]
        tensor = tensor + vector[i].reshape(ax, order = 'F')
    return tensor

## 2nd solution
def vec_combine(vector):
    return (vector[0][:, np.newaxis, np.newaxis] + vector[1][np.newaxis, :, np.newaxis]
            + vector[2][np.newaxis, np.newaxis, :])

In [6]:
vector = []
for i in range(3):
    vector.append(np.array([i + 1 for i in range(i + 2)]))
print(vector)
print(vec_combine(vector))
print()
print(vector[0][1] + vector[1][1] + vector[2][2])
print(vec_combine(vector)[1, 1, 2])

[array([1, 2]), array([1, 2, 3]), array([1, 2, 3, 4])]
[[[3 4 5 6]
  [4 5 6 7]
  [5 6 7 8]]

 [[4 5 6 7]
  [5 6 7 8]
  [6 7 8 9]]]

7
7


### Tensor Unfolding (`ten2mat`)

Using numpy reshape to perform 3rd rank tensor unfold operation. [[**link**](https://stackoverflow.com/questions/49970141/using-numpy-reshape-to-perform-3rd-rank-tensor-unfold-operation)]

In [7]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

In [8]:
X = np.array([[[1, 2, 3, 4], [3, 4, 5, 6]], 
              [[5, 6, 7, 8], [7, 8, 9, 10]], 
              [[9, 10, 11, 12], [11, 12, 13, 14]]])
print('tensor size:')
print(X.shape)
print('original tensor:')
print(X)
print()
print('(1) mode-1 tensor unfolding:')
print(ten2mat(X, 0))
print()
print('(2) mode-2 tensor unfolding:')
print(ten2mat(X, 1))
print()
print('(3) mode-3 tensor unfolding:')
print(ten2mat(X, 2))

tensor size:
(3, 2, 4)
original tensor:
[[[ 1  2  3  4]
  [ 3  4  5  6]]

 [[ 5  6  7  8]
  [ 7  8  9 10]]

 [[ 9 10 11 12]
  [11 12 13 14]]]

(1) mode-1 tensor unfolding:
[[ 1  3  2  4  3  5  4  6]
 [ 5  7  6  8  7  9  8 10]
 [ 9 11 10 12 11 13 12 14]]

(2) mode-2 tensor unfolding:
[[ 1  5  9  2  6 10  3  7 11  4  8 12]
 [ 3  7 11  4  8 12  5  9 13  6 10 14]]

(3) mode-3 tensor unfolding:
[[ 1  5  9  3  7 11]
 [ 2  6 10  4  8 12]
 [ 3  7 11  5  9 13]
 [ 4  8 12  6 10 14]]


In [9]:
def cov_mat(mat, mat_bar):
    mat = mat - mat_bar
    return mat.T @ mat

### Define Performance Metrics

- **RMSE**
- **MAPE**

In [10]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

#### Sample global parameter $\mu$

- Prior distribution:

$$\mu\sim\mathcal{N}(\mu_0,\tau_0^{-1})$$

- Likelihood from:

$$y_{i j t} \sim \mathcal{N}\left(\mu+\phi_{i}+\theta_{j}+\eta_{t}+\sum_{k=1}^{r} u_{i k} v_{j k} x_{t k}, \tau^{-1}\right), \forall(i, j, t)$$

- Posterior distribution:

$$\mu\mid-\sim\mathcal{N}\left(\tilde{\mu},\tilde{\tau}^{-1}\right)$$

where $\tilde{\tau}=\tau_0+\tau|\Omega|$, $\tilde{\mu}=\tilde{\tau}^{-1}\left(\tau_0\mu_0+\tau\sum_{(i,j,t)\in\Omega}\tilde{y}_{ijt}\right)$, and $\tilde{y}_{ijt}=y_{ijt}-(\phi_{i}+\theta_{j}+\eta_{t}+\sum_{k=1}^{r} u_{i k} v_{j k} x_{t k})$.

In [11]:
def sample_global_mu(mu_sparse, pos_obs, tau_eps, tau0 = 1):
    tau_tilde = 1 / (tau_eps * len(pos_obs[0]) + tau0)
    mu_tilde = tau_eps * np.sum(mu_sparse) * tau_tilde
    return np.random.normal(mu_tilde, np.sqrt(tau_tilde))

#### Sample bias vectors $\boldsymbol{\phi},\boldsymbol{\theta},\boldsymbol{\eta}$



In [12]:
def sample_bias_vector(bias_sparse, factor, bias, ind, dim, k, tau_eps, tau0 = 1):
    for k in range(len(dim)):
        idx = tuple(filter(lambda x: x != k, range(len(dim))))
        temp = vector.copy()
        temp[k] = np.zeros((dim[k]))
        tau_tilde = 1 / (tau_eps * bias[k] + tau0)
        mu_tilde = tau_eps * np.sum(ind * (bias_sparse - vec_combine(temp)), axis = idx) * tau_tilde
        vector[k] = np.random.normal(mu_tilde, np.sqrt(tau_tilde))
    return vector

#### Sample factor matrices

In [13]:
def sample_factor(tau_sparse, factor, ind, dim, k, tau_eps, beta0 = 1):
    dim, rank = factor[k].shape
    dim = factor[k].shape[0]
    factor_bar = np.mean(factor[k], axis = 0)
    temp = dim / (dim + beta0)
    var_mu_hyper = temp * factor_bar
    var_W_hyper = inv(np.eye(rank) + cov_mat(factor[k], factor_bar) + temp * beta0 * np.outer(factor_bar, factor_bar))
    var_Lambda_hyper = wishart.rvs(df = dim + rank, scale = var_W_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim + beta0) * var_Lambda_hyper)
    
    idx = list(filter(lambda x: x != k, range(len(factor))))
    var1 = kr_prod(factor[idx[1]], factor[idx[0]]).T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ ten2mat(tau_eps * ind, k).T).reshape([rank, rank, dim]) + var_Lambda_hyper[:, :, np.newaxis]
    var4 = var1 @ ten2mat(tau_sparse, k).T + (var_Lambda_hyper @ var_mu_hyper)[:, np.newaxis]
    for i in range(dim):
        factor[k][i, :] = mvnrnd_pre(solve(var3[:, :, i], var4[:, i]), var3[:, :, i])
    return factor[k]

#### Sample precision $\tau$

In [14]:
def sample_precision_tau(error_tensor, pos_obs):
    var_alpha = 1e-6 + 0.5 * len(pos_obs[0])
    var_beta = 1e-6 + 0.5 * np.linalg.norm(error_tensor, 2) ** 2
    return np.random.gamma(var_alpha, 1 / var_beta)

#### BATF with Gibbs sampling

In [15]:
def BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter):
    """Bayesian Augmented Tensor Factorization (BATF) with Gibbs sampling."""

    dim = np.array(sparse_tensor.shape)
    rank = factor[0].shape[1]
    if np.isnan(sparse_tensor).any() == False:
        ind = sparse_tensor != 0
        pos_obs = np.where(ind)
        pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    elif np.isnan(sparse_tensor).any() == True:
        pos_test = np.where((dense_tensor != 0) & (np.isnan(sparse_tensor)))
        ind = ~np.isnan(sparse_tensor)
        pos_obs = np.where(ind)
        sparse_tensor[np.isnan(sparse_tensor)] = 0
    num_obs = len(pos_obs[0])
    dense_test = dense_tensor[pos_test]
    del dense_tensor

    show_iter = 200
    tau_eps = 1
    bias = []
    for k in range(len(dim)):
        idx = tuple(filter(lambda x: x != k, range(len(dim))))
        bias.append(np.sum(ind, axis = idx))
    temp = cp_combine(factor)
    temp_hat = np.zeros(len(pos_test[0]))
    tensor_hat_plus = np.zeros(dim)
    for it in range(burn_iter + gibbs_iter):
        temp = sparse_tensor - temp
        mu_glb = sample_global_mu(temp[pos_obs] - vec_combine(vector)[pos_obs], pos_obs, tau_eps)
        vector = sample_bias_vector(temp - mu_glb, factor, bias, ind, dim, k, tau_eps)
        del temp
        tau_sparse = tau_eps * ind * (sparse_tensor - mu_glb - vec_combine(vector))
        for k in range(len(dim)):
            factor[k] = sample_factor(tau_sparse, factor, ind, dim, k, tau_eps)
        temp = cp_combine(factor)
        tensor_hat = mu_glb + vec_combine(vector) + temp
        temp_hat += tensor_hat[pos_test]
        tau_eps = sample_precision_tau(sparse_tensor[pos_obs] - tensor_hat[pos_obs], pos_obs)
        if it + 1 > burn_iter:
            tensor_hat_plus += tensor_hat
        if (it + 1) % show_iter == 0 and it < burn_iter:
            temp_hat = temp_hat / show_iter
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            temp_hat = np.zeros(len(pos_test[0]))
            print()
    tensor_hat = tensor_hat_plus / gibbs_iter
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_test, tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_test, tensor_hat[pos_test])))
    print()

    return tensor_hat, mu_glb, vector, factor

## Data Organization

### 1) Matrix Structure

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{f},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We express spatio-temporal dataset as a matrix $Y\in\mathbb{R}^{m\times f}$ with $m$ rows (e.g., locations) and $f$ columns (e.g., discrete time intervals),

$$Y=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{m1} & y_{m2} & \cdots & y_{mf} \\ \end{array} \right]\in\mathbb{R}^{m\times f}.$$

### 2) Tensor Structure

We consider a dataset of $m$ discrete time series $\boldsymbol{y}_{i}\in\mathbb{R}^{nf},i\in\left\{1,2,...,m\right\}$. The time series may have missing elements. We partition each time series into intervals of predifined length $f$. We express each partitioned time series as a matrix $Y_{i}$ with $n$ rows (e.g., days) and $f$ columns (e.g., discrete time intervals per day),

$$Y_{i}=\left[ \begin{array}{cccc} y_{11} & y_{12} & \cdots & y_{1f} \\ y_{21} & y_{22} & \cdots & y_{2f} \\ \vdots & \vdots & \ddots & \vdots \\ y_{n1} & y_{n2} & \cdots & y_{nf} \\ \end{array} \right]\in\mathbb{R}^{n\times f},i=1,2,...,m,$$

therefore, the resulting structure is a tensor $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$.

## Evaluation on Guangzhou Speed Data

**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Random missing (RM)
- 40% missing rate


**Model setting**:

- Low rank: 80
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [17]:
import time
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.4 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 80
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.08303
RMSE: 3.58407

Iter: 400
MAPE: 0.083102
RMSE: 3.59169

Iter: 600
MAPE: 0.0830566
RMSE: 3.59025

Iter: 800
MAPE: 0.0829992
RMSE: 3.58897

Iter: 1000
MAPE: 0.0829348
RMSE: 3.58643

Imputation MAPE: 0.0828811
Imputation RMSE: 3.5845

Running time: 3592 seconds


**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Random missing (RM)
- 60% missing rate


**Model setting**:

- Low rank: 80
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [19]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.6 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 80
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0844221
RMSE: 3.63857

Iter: 400
MAPE: 0.0843421
RMSE: 3.64409

Iter: 600
MAPE: 0.084293
RMSE: 3.64176

Iter: 800
MAPE: 0.0842479
RMSE: 3.63975

Iter: 1000
MAPE: 0.084262
RMSE: 3.64037

Imputation MAPE: 0.0842613
Imputation RMSE: 3.63993

Running time: 3612 seconds


**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Non-random missing (NM)
- 40% missing rate


**Model setting**:

- Low rank: 10
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [21]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.4 # Non-random missing (NM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1])[:, :, np.newaxis] + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.101644
RMSE: 4.29233

Iter: 400
MAPE: 0.10161
RMSE: 4.30334

Iter: 600
MAPE: 0.101477
RMSE: 4.30121

Iter: 800
MAPE: 0.101412
RMSE: 4.29953

Iter: 1000
MAPE: 0.101376
RMSE: 4.29767

Imputation MAPE: 0.101357
Imputation RMSE: 4.29761

Running time: 285 seconds


## Evaluation on Birmingham Parking Data

**Scenario setting**:

- Tensor size: $30\times 77\times 18$ (parking slot, day, time of day)
- Random missing (RM)
- 40% missing rate


**Model setting**:

- Low rank: 20
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [23]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.4 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0766492
RMSE: 28.5921

Iter: 400
MAPE: 0.0729341
RMSE: 26.6797

Iter: 600
MAPE: 0.0727347
RMSE: 26.0331

Iter: 800
MAPE: 0.0736088
RMSE: 25.7955

Iter: 1000
MAPE: 0.0726944
RMSE: 25.2289

Imputation MAPE: 0.0732128
Imputation RMSE: 25.2056

Running time: 23 seconds


**Scenario setting**:

- Tensor size: $30\times 77\times 18$ (parking slot, day, time of day)
- Random missing (RM)
- 60% missing rate


**Model setting**:

- Low rank: 20
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [25]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.6 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0877826
RMSE: 32.2015

Iter: 400
MAPE: 0.0881449
RMSE: 31.8352

Iter: 600
MAPE: 0.0866449
RMSE: 31.4774

Iter: 800
MAPE: 0.085667
RMSE: 31.2294

Iter: 1000
MAPE: 0.0840369
RMSE: 30.5517

Imputation MAPE: 0.0838647
Imputation RMSE: 29.7391

Running time: 28 seconds


**Scenario setting**:

- Tensor size: $30\times 77\times 18$ (parking slot, day, time of day)
- Non-random missing (NM)
- 40% missing rate


**Model setting**:

- Low rank: 20
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [27]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Birmingham-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.4 # Non-random missing (NM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1])[:, :, np.newaxis] + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.144786
RMSE: 79.5898

Iter: 400
MAPE: 0.167861
RMSE: 93.8396

Iter: 600
MAPE: 0.179682
RMSE: 105.395

Iter: 800
MAPE: 0.178547
RMSE: 109.112

Iter: 1000
MAPE: 0.17795
RMSE: 109.826

Imputation MAPE: 0.177455
Imputation RMSE: 113.002

Running time: 20 seconds


## Evaluation on Hangzhou Flow Data

**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Random missing (RM)
- 40% missing rate


**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [29]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.4 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.231115
RMSE: 30.3267

Iter: 400
MAPE: 0.228563
RMSE: 36.0327

Iter: 600
MAPE: 0.229881
RMSE: 39.5089

Iter: 800
MAPE: 0.204963
RMSE: 45.2299

Iter: 1000
MAPE: 0.199239
RMSE: 45.8608

Imputation MAPE: 0.198827
Imputation RMSE: 44.8775

Running time: 140 seconds


**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Random missing (RM)
- 60% missing rate


**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [31]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.6 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.238708
RMSE: 32.1782

Iter: 400
MAPE: 0.240156
RMSE: 33.2013

Iter: 600
MAPE: 0.215423
RMSE: 34.7814

Iter: 800
MAPE: 0.205286
RMSE: 37.8658

Iter: 1000
MAPE: 0.205756
RMSE: 38.3135

Imputation MAPE: 0.204502
Imputation RMSE: 38.8262

Running time: 139 seconds


**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Non-random missing (NM)
- 40% missing rate


**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [33]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
missing_rate = 0.4 # Non-random missing (NM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1])[:, :, np.newaxis] + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.249926
RMSE: 30.0841

Iter: 400
MAPE: 0.241747
RMSE: 34.1185

Iter: 600
MAPE: 0.236539
RMSE: 35.6748

Iter: 800
MAPE: 0.216268
RMSE: 36.2932

Iter: 1000
MAPE: 0.211793
RMSE: 38.6953

Imputation MAPE: 0.212244
Imputation RMSE: 40.4924

Running time: 129 seconds


## Evaluation on Seattle Speed Data

**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Random missing (RM)
- 40% missing rate


**Model setting**:

- Low rank: 50
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [35]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Seattle-data-set/tensor.npz')['arr_0']
dim = dense_tensor.shape
missing_rate = 0.4 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 50
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0745654
RMSE: 4.49519

Iter: 400
MAPE: 0.0744064
RMSE: 4.48914

Iter: 600
MAPE: 0.0742243
RMSE: 4.48193

Iter: 800
MAPE: 0.0741202
RMSE: 4.47552

Iter: 1000
MAPE: 0.0740641
RMSE: 4.47165

Imputation MAPE: 0.0740247
Imputation RMSE: 4.46906

Running time: 2895 seconds


**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Random missing (RM)
- 60% missing rate


**Model setting**:

- Low rank: 50
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [37]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Seattle-data-set/tensor.npz')['arr_0']
dim = dense_tensor.shape
missing_rate = 0.6 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 50
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0753578
RMSE: 4.52268

Iter: 400
MAPE: 0.0749893
RMSE: 4.51293

Iter: 600
MAPE: 0.0747484
RMSE: 4.50605

Iter: 800
MAPE: 0.0746789
RMSE: 4.5049

Iter: 1000
MAPE: 0.0746148
RMSE: 4.50328

Imputation MAPE: 0.0745356
Imputation RMSE: 4.50221

Running time: 2864 seconds


**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Non-random missing (NM)
- 40% missing rate


**Model setting**:

- Low rank: 10
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [39]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/Seattle-data-set/tensor.npz')['arr_0']
dim = dense_tensor.shape
missing_rate = 0.4 # Non-random missing (NM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1])[:, :, np.newaxis] + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0986925
RMSE: 5.64095

Iter: 400
MAPE: 0.0998044
RMSE: 5.71104

Iter: 600
MAPE: 0.0998826
RMSE: 5.70964

Iter: 800
MAPE: 0.0994049
RMSE: 5.68442

Iter: 1000
MAPE: 0.099428
RMSE: 5.68895

Imputation MAPE: 0.0993864
Imputation RMSE: 5.68241

Running time: 469 seconds


## Evaluation on London Movement Speed Data

**Scenario setting**:

- Tensor size: $35912\times 30\times 24$ (road segment, day, time of day)
- Random missing (RM)
- 40% missing rate


In [16]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.4

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

dense_tensor = dense_mat.reshape([dense_mat.shape[0], 30, 24])
sparse_tensor = sparse_mat.reshape([sparse_mat.shape[0], 30, 24])
del dense_mat, sparse_mat, binary_mat, random_mat

**Model setting**:

- Low rank: 20
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [17]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0924598
RMSE: 2.2497

Iter: 400
MAPE: 0.0922938
RMSE: 2.24709

Iter: 600
MAPE: 0.0921999
RMSE: 2.24483

Iter: 800
MAPE: 0.0921524
RMSE: 2.24389

Iter: 1000
MAPE: 0.0921205
RMSE: 2.24329

Imputation MAPE: 0.0920917
Imputation RMSE: 2.24285

Running time: 15725 seconds


**Scenario setting**:

- Tensor size: $35912\times 30\times 24$ (road segment, day, time of day)
- Random missing (RM)
- 60% missing rate


In [18]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.6

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

dense_tensor = dense_mat.reshape([dense_mat.shape[0], 30, 24])
sparse_tensor = sparse_mat.reshape([sparse_mat.shape[0], 30, 24])
del dense_mat, sparse_mat, binary_mat, random_mat

**Model setting**:

- Low rank: 20
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [19]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.093838
RMSE: 2.27918

Iter: 400
MAPE: 0.0936782
RMSE: 2.2763

Iter: 600
MAPE: 0.0935403
RMSE: 2.2732

Iter: 800
MAPE: 0.0934167
RMSE: 2.27159

Iter: 1000
MAPE: 0.0933518
RMSE: 2.27042

Imputation MAPE: 0.0933077
Imputation RMSE: 2.26969

Running time: 14963 seconds


**Scenario setting**:

- Tensor size: $35912\times 30\times 24$ (road segment, day, time of day)
- Non-random missing (NM)
- 40% missing rate


In [20]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.4

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Non-random missing (NM)
binary_mat = np.zeros(dense_mat.shape)
random_mat = np.random.rand(dense_mat.shape[0], 30)
for i1 in range(dense_mat.shape[0]):
    for i2 in range(30):
        binary_mat[i1, i2 * 24 : (i2 + 1) * 24] = np.round(random_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

dense_tensor = dense_mat.reshape([dense_mat.shape[0], 30, 24])
sparse_tensor = sparse_mat.reshape([sparse_mat.shape[0], 30, 24])
del dense_mat, sparse_mat, binary_mat, random_mat

**Model setting**:

- Low rank: 20
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [21]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.095513
RMSE: 2.32949

Iter: 400
MAPE: 0.0955237
RMSE: 2.3318

Iter: 600
MAPE: 0.0955272
RMSE: 2.33183

Iter: 800
MAPE: 0.0955202
RMSE: 2.33174

Iter: 1000
MAPE: 0.0955029
RMSE: 2.33134

Imputation MAPE: 0.0954908
Imputation RMSE: 2.33104

Running time: 14209 seconds


## Evaluation on New York Taxi Data

**Scenario setting**:

- Tensor size: $30\times 30\times 1464$ (origin, destination, time)
- Random missing (RM)
- 40% missing rate


**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [17]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
dim = dense_tensor.shape
missing_rate = 0.4 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.496458
RMSE: 4.86813

Iter: 400
MAPE: 0.491284
RMSE: 4.92522

Iter: 600
MAPE: 0.494559
RMSE: 4.91508

Iter: 800
MAPE: 0.497879
RMSE: 4.8992

Iter: 1000
MAPE: 0.494153
RMSE: 4.90593

Imputation MAPE: 0.493206
Imputation RMSE: 4.90428

Running time: 2359 seconds


**Scenario setting**:

- Tensor size: $30\times 30\times 1464$ (origin, destination, time)
- Random missing (RM)
- 60% missing rate


**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [19]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
dim = dense_tensor.shape
missing_rate = 0.6 # Random missing (RM)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim[0], dim[1], dim[2]) + 0.5 - missing_rate)

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.50914
RMSE: 5.00745

Iter: 400
MAPE: 0.502967
RMSE: 5.08066

Iter: 600
MAPE: 0.503814
RMSE: 5.10919

Iter: 800
MAPE: 0.502028
RMSE: 5.10919

Iter: 1000
MAPE: 0.502449
RMSE: 5.11802

Imputation MAPE: 0.501079
Imputation RMSE: 5.11917

Running time: 2386 seconds


**Scenario setting**:

- Tensor size: $30\times 30\times 1464$ (origin, destination, time)
- Non-random missing (NM)
- 40% missing rate


**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [21]:
import time
import scipy.io
import numpy as np
np.random.seed(1000)

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor']
dim = dense_tensor.shape
nm_tensor = np.random.rand(dim[0], dim[1], dim[2])
missing_rate = 0.4 # Non-random missing (NM)
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dim[0]):
    for i2 in range(dim[1]):
        for i3 in range(61):
            binary_tensor[i1, i2, i3 * 24 : (i3 + 1) * 24] = np.round(nm_tensor[i1, i2, i3] + 0.5 - missing_rate)
sparse_tensor = dense_tensor * binary_tensor

start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.565532
RMSE: 4.84217

Iter: 400
MAPE: 0.559177
RMSE: 4.89503

Iter: 600
MAPE: 0.556918
RMSE: 4.89814

Iter: 800
MAPE: 0.55587
RMSE: 4.88751

Iter: 1000
MAPE: 0.547338
RMSE: 4.87227

Imputation MAPE: 0.547406
Imputation RMSE: 4.86312

Running time: 2120 seconds


## Evaluation on Pacific Temperature Data

**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (grid, grid, time)
- Random missing (RM)
- 40% missing rate


In [16]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
pos = np.where(dense_tensor[:, 0, :] > 50)
dense_tensor[pos[0], :, pos[1]] = 0
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], dense_tensor.shape[2])
missing_rate = 0.4

## Random missing (RM)
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [17]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0150289
RMSE: 0.50347

Iter: 400
MAPE: 0.0148072
RMSE: 0.493748

Iter: 600
MAPE: 0.0145197
RMSE: 0.483689

Iter: 800
MAPE: 0.0144342
RMSE: 0.480458

Iter: 1000
MAPE: 0.014325
RMSE: 0.477329

Imputation MAPE: 0.0142515
Imputation RMSE: 0.475171

Running time: 582 seconds


**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (grid, grid, time)
- Random missing (RM)
- 60% missing rate


In [18]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
pos = np.where(dense_tensor[:, 0, :] > 50)
dense_tensor[pos[0], :, pos[1]] = 0
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], dense_tensor.shape[2])
missing_rate = 0.6

## Random missing (RM)
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [19]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0151992
RMSE: 0.510084

Iter: 400
MAPE: 0.0148552
RMSE: 0.497413

Iter: 600
MAPE: 0.0146831
RMSE: 0.491068

Iter: 800
MAPE: 0.0145299
RMSE: 0.48578

Iter: 1000
MAPE: 0.0144233
RMSE: 0.481933

Imputation MAPE: 0.0143321
Imputation RMSE: 0.478828

Running time: 564 seconds


**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (grid, grid, time)
- Non-random missing (NM)
- 40% missing rate


In [20]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
pos = np.where(dense_tensor[:, 0, :] > 50)
dense_tensor[pos[0], :, pos[1]] = 0
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], int(dense_tensor.shape[2] / 3))
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        for i3 in range(int(dense_tensor.shape[2] / 3)):
            binary_tensor[i1, i2, i3 * 3 : (i3 + 1) * 3] = np.round(random_tensor[i1, i2, i3] + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [21]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 30
vector = []
factor = []
for k in range(len(dim)):
    vector.append(0.1 * np.random.randn(dim[k],))
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BATF_Gibbs(dense_tensor, sparse_tensor, vector, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0151078
RMSE: 0.506104

Iter: 400
MAPE: 0.0149149
RMSE: 0.49768

Iter: 600
MAPE: 0.0147768
RMSE: 0.492178

Iter: 800
MAPE: 0.0146589
RMSE: 0.487993

Iter: 1000
MAPE: 0.0145806
RMSE: 0.485344

Imputation MAPE: 0.0145099
Imputation RMSE: 0.483019

Running time: 552 seconds


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>